<a href="https://colab.research.google.com/github/jaybirrd/nlp_name_classifier/blob/master/name_classifier_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP FROM SCRATCH: CLASSIFYING NAMES WITH A CHARACTER-LEVEL RNN

# clone data

In [0]:
!git clone https://github.com/jaybirrd/nlp_name_classifier.git

Cloning into 'nlp_name_classifier'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 49 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), done.


# prepare the data

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def find_files(path):
  return glob.glob(path)

#print(find_files('nlp_name_classifier/data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicode_to_ascii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
  )

#print(unicode_to_ascii('Ślusàrski'))

category_lines = {}
all_categories = []

def read_lines(filename):
  lines = open(filename, encoding='utf-8').read().strip().split('\n')
  return [unicode_to_ascii(line) for line in lines]

#FUTURE DEV: put a popup here to select different datasets to analyze

for filename in find_files('nlp_name_classifier/data/names/*.txt'):
  category = os.path.splitext(os.path.basename(filename))[0]
  all_categories.append(category)
  lines = read_lines(filename)
  category_lines[category] = lines

n_categories = len(all_categories)

#print(category_lines['Italian'][:5])

['nlp_name_classifier/data/names/Polish.txt', 'nlp_name_classifier/data/names/French.txt', 'nlp_name_classifier/data/names/Vietnamese.txt', 'nlp_name_classifier/data/names/Chinese.txt', 'nlp_name_classifier/data/names/German.txt', 'nlp_name_classifier/data/names/Scottish.txt', 'nlp_name_classifier/data/names/Portuguese.txt', 'nlp_name_classifier/data/names/Japanese.txt', 'nlp_name_classifier/data/names/English.txt', 'nlp_name_classifier/data/names/Korean.txt', 'nlp_name_classifier/data/names/Spanish.txt', 'nlp_name_classifier/data/names/Italian.txt', 'nlp_name_classifier/data/names/Irish.txt', 'nlp_name_classifier/data/names/Greek.txt', 'nlp_name_classifier/data/names/Czech.txt', 'nlp_name_classifier/data/names/Russian.txt', 'nlp_name_classifier/data/names/Arabic.txt', 'nlp_name_classifier/data/names/Dutch.txt']
Slusarski
['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


# turn names into tensors

In [0]:
import torch

def letter_to_index(letter):
  return all_letters.find(letter)

#turn a letter into a <1 x n_letters> Tensor
def letter_to_tensor(letter):
  tensor = torch.zeros(1,n_letters)
  tensor[0][letter_to_index(letter)] = 1
  return tensor

#turn lines into a <line length x 1 x n_letters> Tensor
def line_to_tensor(line):
  tensor = torch.zeros(len(line),1,n_letters)
  for i, letter in enumerate(line):
    tensor[i][0][letter_to_index(letter)] = 1
    return tensor

#print(line_to_tensor('James').size())
#print(letter_to_tensor('J'))

torch.Size([5, 1, 57])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])


# creating the network